# pFound

### Import Modules

In [11]:
import pandas as pd

### Reading data

In [12]:
qid_query = pd.read_csv("open_task/qid_query.tsv", sep="\t", names=["qid", "query"])
qid_url_rating = pd.read_csv("open_task/qid_url_rating.tsv", sep="\t", names=["qid", "url", "rating"])
hostid_url = pd.read_csv("open_task/hostid_url.tsv", sep="\t", names=["hostid", "url"])

### Calculate pFound

In [13]:
def pfound_by_group(group: pd.DataFrame):
    qid_group = group.sort_values(ascending=False).reset_index().rename(columns={group.name: 'rating'})
    qid_group['1_pRel'] = 1 - qid_group['rating']
    qid_group['1_pBreak'] = 1 - pBreak
    qid_group['cum_1_pRel'] = qid_group['1_pRel'].cumprod().shift(1)
    qid_group['cum_1_pBreak'] = qid_group['1_pBreak'].cumprod().shift(1)
    qid_group['plook'] = qid_group['cum_1_pBreak'] * qid_group['cum_1_pRel']
    qid_group['plook'].fillna(1, inplace=True)
    qid_group['plook_prod_rel'] = qid_group['plook'] * qid_group['rating']
    return qid_group['plook_prod_rel'].sum()

In [14]:
def get_pfound(df: pd.DataFrame) -> pd.DataFrame:
    df = df.groupby(['qid', 'hostid']).max().reset_index()
    df['pfound'] = df.groupby('qid')['rating'].transform(pfound_by_group)
    df = df.groupby('qid')['pfound'].max()
    df = df.mean()
    return df

In [10]:
qid_url_rating_hostid = pd.merge(qid_url_rating, hostid_url, on="url")
pBreak = 0.15
pfound = get_pfound(qid_url_rating_hostid)
print(pfound)

0.4681776808809003
